In [1]:
import pandas as pd
from src.geo_tools import generate_google_maps_html
import pandasai as pai
from src.tools import extract_data_intent, safe_dataframe_tool, get_openai_llm
from src.classifiers import llm_classifier
import json

Dataset loaded successfully.


In [2]:
"Step 1 - original question"

QUERY = "show on map furnished single room properties in westmister"

In [3]:
"Step 2 - What is the data needed to answer the question?"

processed_query = extract_data_intent.invoke(QUERY)

print(processed_query)

Compare the price_gbp for properties with bedrooms equal to 1 and property_type containing "flat", grouped by borough in London.


In [23]:
"Step 3 - What to do with the above data either [print, plot or map]?"

print(llm_classifier(QUERY))

geospatial_plot
geospatial_plot


In [11]:
# "Step 4 - get the data asked for in Step 2 by pandas ai"
#
# ans = safe_dataframe_tool.invoke(processed_query)
#
# print("Sucsees = ", json.loads(ans)['success'])
# print("Result = ", pd.DataFrame(json.loads(ans)['result']))

In [3]:
ans = safe_dataframe_tool.invoke(QUERY)


print("Sucsees = ", json.loads(ans)['success'])
print("Result = ", pd.DataFrame(json.loads(ans)['result']))

Dataset loaded successfully.
{'type': 'dataframe', 'value':             id                                              title  \
0    147560057  1 bedroom flat share for rent in Room 5, 58A C...   
1    149396807  1 bedroom apartment for rent in Dunraven Stree...   
2    156112556  1 bedroom flat for rent in Catherine Place, \n...   
3    147560597  1 bedroom flat share for rent in Room 5, 52 Ch...   
4    154843757  1 bedroom apartment for rent in Hay Hill, Mayf...   
..         ...                                                ...   
769  156424463  1 bedroom apartment for rent in Bentinck House...   
770  156517772  1 bedroom flat for rent in Medway Street, Vict...   
771  155848805  1 bedroom flat for rent in North Audley Street...   
772    5688648  1 bedroom ground floor flat for rent in Bell S...   
773  145972292  1 bedroom apartment for rent in Welbeck Street...   

                                   display_address postcode_outcode  \
0                 Room 5, 58A Chippenham

In [11]:
result = json.loads(ans)['result']

In [12]:
result

[{'id': 155473763,
  'title': '1 bedroom house share for rent in Waterhead, Varndell Street, NW1',
  'display_address': 'Waterhead, Varndell Street, NW1',
  'postcode_outcode': 'NW1',
  'postcode_incode': '3RR',
  'price_gbp': 899,
  'latitude': 51.529281,
  'longitude': -0.139209},
 {'id': 87059196,
  'title': '1 bedroom house share for rent in Harmood Street, Chalk Farm, NW1',
  'display_address': 'Harmood Street, Chalk Farm, NW1',
  'postcode_outcode': 'NW1',
  'postcode_incode': '8DY',
  'price_gbp': 899,
  'latitude': 51.544647,
  'longitude': -0.14774},
 {'id': 155616965,
  'title': '1 bedroom house share for rent in Chalton Street, Euston, , NW1',
  'display_address': 'Chalton Street, Euston, ',
  'postcode_outcode': 'NW1',
  'postcode_incode': '1HH',
  'price_gbp': 899,
  'latitude': 51.530622,
  'longitude': -0.13133},
 {'id': 155857817,
  'title': "1 bedroom house share for rent in Stanhope Street, Regent's Park, NW1",
  'display_address': "Stanhope Street, Regent's Park, NW1

In [4]:
html = generate_google_maps_html(json.loads(ans)['result'])

In [5]:
html

'<html><body><h1>Too many properties to display. Please refine your search.</h1></body></html>'

In [6]:
with open("properties_westminster_large_number.html", "w", encoding="utf-8") as f:
    f.write(html)

print("Saved as properties_map.html")

Saved as properties_map.html


In [7]:
from prompts.tool_prompts import get_plotly_code_prompt

In [8]:
prompt = get_plotly_code_prompt(QUERY,data)

In [9]:
print(prompt)


You are an assistant that generates **only Python code** using Plotly (not matplotlib).

The dataset is already available in memory as a variable named `df`.

Dataset columns: borough, avg_price
Sample rows (first 5, for reference only — do NOT hardcode them): [{'borough': 'Kensington and Chelsea', 'avg_price': 3628.487394958}, {'borough': 'Westminster', 'avg_price': 3459.3021201413}, {'borough': 'City of London', 'avg_price': 3157.4137931034}, {'borough': 'Camden', 'avg_price': 2582.2489270386}, {'borough': 'Hammersmith and Fulham', 'avg_price': 2424.3733905579}]

User request: "compare prices per each london borough for 1 bedroom flat"

Instructions:
- Always use the full dataset `df` (not just the sample).
- Do not recreate or hardcode the dataset.
- If there is only one row, still plot it (e.g., single bar or single point).
- Choose the most appropriate plot type (bar, line, scatter, histogram, etc.).
- Return only valid Python code, wrapped like this:

```python
# code here



In [10]:
from openai.types.chat import ChatCompletionSystemMessageParam, ChatCompletionUserMessageParam

official_ai = get_openai_llm()
response = official_ai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=
        [
            ChatCompletionSystemMessageParam(role="system",
                                             content="You create Plotly code based on user_input and data"),
            ChatCompletionUserMessageParam(role="user", content=prompt)
        ],
        max_tokens=400
    )
    # Extract code
raw_response = response.choices[0].message.content

In [11]:
print(raw_response)

```python
import plotly.express as px

fig = px.bar(df, x='borough', y='avg_price',
             title='Average Price per 1 Bedroom Flat by London Borough',
             labels={'borough': 'London Borough', 'avg_price': 'Average Price (£)'},
             text=df['avg_price'].round(0))

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, yaxis_title='Average Price (£)', yaxis=dict(tickformat=",.0f"))
fig.show()
```


In [12]:
from src.tools import extract_python_code

In [13]:
code = extract_python_code(raw_response)

print(code)

import plotly.express as px

fig = px.bar(df, x='borough', y='avg_price',
             title='Average Price per 1 Bedroom Flat by London Borough',
             labels={'borough': 'London Borough', 'avg_price': 'Average Price (£)'},
             text=df['avg_price'].round(0))

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, yaxis_title='Average Price (£)', yaxis=dict(tickformat=",.0f"))
fig.show()


In [14]:
code = code.replace("fig.show()", "")
code += "\nst.plotly_chart(fig, use_container_width=True)"

print(code)

import plotly.express as px

fig = px.bar(df, x='borough', y='avg_price',
             title='Average Price per 1 Bedroom Flat by London Borough',
             labels={'borough': 'London Borough', 'avg_price': 'Average Price (£)'},
             text=df['avg_price'].round(0))

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, yaxis_title='Average Price (£)', yaxis=dict(tickformat=",.0f"))

st.plotly_chart(fig, use_container_width=True)


In [11]:
print(pd.DataFrame(json.loads(safe_dataframe_tool.invoke(QUERY))['result']))

Dataset loaded successfully.
                   borough    avg_price
0           City of London  3919.207792
1              Westminster  3359.210151
2   Kensington and Chelsea  3200.649524
3                   Camden  2537.114035
4               Wandsworth  2414.468085
5   Hammersmith and Fulham  2393.431915
6            Tower Hamlets  2294.065121
7                Southwark  2219.508621
8                  Hackney  2192.873377
9                Islington  2154.578406
10                 Lambeth  2095.029851
11    Richmond upon Thames  2021.737931
12                  Newham  1896.700599
13                   Brent  1720.505338
14                Haringey  1701.506623
15               Greenwich  1661.082192
16                Hounslow  1634.705645
17                  Merton  1565.407080
18                  Ealing  1564.715517
19                Lewisham  1535.686207
20                  Barnet  1512.812500
21    Kingston upon Thames  1466.611765
22          Waltham Forest  1428.560440
23         